In [11]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain

from langchain.prompts import PromptTemplate

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory

from langchain.agents import load_tools

In [12]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [13]:
# llm = OpenAI(temperature=0, verbose=True)
llm = ChatOpenAI(model_name='gpt-4',temperature=0)

In [14]:
tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [15]:
from langchain.tools import Tool, BaseTool
from pydantic.v1 import BaseModel, Field
import math
from scipy.stats import norm

def rw_corp(Input_String: str) -> str:

    """ 
    calculates the risk weight from input parameters 
    PD, LGD, MATURITY, F_LARGE_FIN and SIZE which are fed into the formula in this particular order 
    """
    
    PD_s, LGD_s, MATURITY_s, SIZE_s, F_LARGE_FIN_s = Input_String.split(",")

    PD = float(eval(PD_s.split(":")[1].strip()))
    LGD = float(eval(LGD_s.split(":")[1].strip()))
    MATURITY = float(eval(MATURITY_s.split(":")[1].strip()))
    SIZE = float(eval(SIZE_s.split(":")[1].strip()))
    F_LARGE_FIN = (F_LARGE_FIN_s.split(":")[1].strip())
       
    pd_final = max(0.0003, PD)
    size_final = max(5, min(SIZE, 50))    
    r0 = (0.12 * (1.0 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0))) + \
        (0.24 * (1.0 - (1 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0)))) - \
        (0.04 * (1.0 - (size_final - 5.0) / 45.0))
    if F_LARGE_FIN == 'Y':
        r = r0 * 1.25
    else:
        r = r0
    b = (0.11852 - 0.05478 * math.log(pd_final)) ** 2
    ma = ((1 - 1.5 * b) ** -1) * (1 + (MATURITY - 2.5) * b)    
    rw = ((LGD * norm.cdf((1 - r) ** -0.5 * norm.ppf(pd_final) + (r / (1 - r)) ** 0.5 * norm.ppf(0.999)) - pd_final * LGD) * ma) * 12.5 * 1.06
    return rw  


class RWInput(BaseModel):
    Input_String: str = Field(description='This is a string that contains values for the input parameters \
                              PD, LGD, MATURITY, SIZE and F_LARGE_FIN which are fed into the formula in this particular order ')
    
RWTool = Tool.from_function(
    func=rw_corp,
    name="RWTool",
    description="""
    This is a custom tool that calculates the risk weight from a set of input parameters:
        PD - Probability of Default,
        LGD - Loss Given Default,
        MATURITY - Remaining maturity of the loan in years,
        SIZE - The size of the client in MEUR, usually this is the client's turnover, 
        F_LARGE_FIN - If 'Y' the client is a Large Financial Institution        
        """,
    args_schema=RWInput
)

In [16]:
tools.append(RWTool)

In [17]:
"""SQL agent."""
from typing import Any, Dict, List, Optional, Sequence

from langchain.agents.agent import AgentExecutor, BaseSingleActionAgent
from langchain.agents.agent_toolkits.sql.prompt import (
    SQL_FUNCTIONS_SUFFIX,
    SQL_PREFIX,
    SQL_SUFFIX,
)
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.callbacks.base import BaseCallbackManager
from langchain.chains.llm import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema.language_model import BaseLanguageModel
from langchain.schema.messages import AIMessage, SystemMessage
from langchain.tools import BaseTool

def create_sql_agent(
    llm: BaseLanguageModel,
    toolkit: SQLDatabaseToolkit,
    agent_type: AgentType = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callback_manager: Optional[BaseCallbackManager] = None,
    prefix: str = SQL_PREFIX,
    suffix: Optional[str] = None,
    format_instructions: str = FORMAT_INSTRUCTIONS,
    input_variables: Optional[List[str]] = None,
    top_k: int = 10,
    max_iterations: Optional[int] = 15,
    max_execution_time: Optional[float] = None,
    early_stopping_method: str = "force",
    verbose: bool = False,
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    extra_tools: Sequence[BaseTool] = (),
    memory: Optional[ConversationBufferMemory] = None,
    **kwargs: Any,
) -> AgentExecutor:
    """Construct an SQL agent from an LLM and tools."""
    tools = toolkit.get_tools() + list(extra_tools)
    prefix = prefix.format(dialect=toolkit.dialect, top_k=top_k)
    agent: BaseSingleActionAgent

    if agent_type == AgentType.ZERO_SHOT_REACT_DESCRIPTION:
        prompt = ZeroShotAgent.create_prompt(
            tools,
            prefix=prefix,
            suffix=suffix or SQL_SUFFIX,
            format_instructions=format_instructions,
            input_variables=input_variables,
        )
        llm_chain = LLMChain(
            llm=llm,
            prompt=prompt,
            callback_manager=callback_manager,
        )
        tool_names = [tool.name for tool in tools]
        agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names, **kwargs)

    elif agent_type == AgentType.OPENAI_FUNCTIONS:
        messages = [
            SystemMessage(content=prefix),
            HumanMessagePromptTemplate.from_template("{input}"),
            AIMessage(content=suffix or SQL_FUNCTIONS_SUFFIX),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
        input_variables = ["input", "agent_scratchpad"]
        _prompt = ChatPromptTemplate(input_variables=input_variables, messages=messages)

        agent = OpenAIFunctionsAgent(
            llm=llm,
            prompt=_prompt,
            tools=tools,
            callback_manager=callback_manager,
            **kwargs,
        )
    else:
        raise ValueError(f"Agent type {agent_type} not supported at the moment.")

    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        callback_manager=callback_manager,
        verbose=verbose,
        max_iterations=max_iterations,
        max_execution_time=max_execution_time,
        early_stopping_method=early_stopping_method,
        memory=memory,
        **(agent_executor_kwargs or {}),
    )


In [18]:
db = SQLDatabase.from_uri("sqlite:///./portfolio_data.db",
                         include_tables=['corporate_portfolio'],
                         sample_rows_in_table_info=2
)

In [26]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent='conversational-react-description', 
    extra_tools=tools,
    memory=memory 
)

### sql

In [20]:
output = agent.run("What are total RWA and EAD for reporting date 31-12-2022?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: corporate_portfolio
Thought:The corporate_portfolio table might contain the information I need. I should check its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio"
Observation: 
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0.00

In [22]:
output = agent.run("What is the change in total RWA from reporting date 2022-12-31 to reporting date 2023-03-31?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: corporate_portfolio
Thought:The corporate_portfolio table might contain the information I need. I should check its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio"
Observation: 
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0.00

In [23]:
output = agent.run("What is the total exposure of only those loan ids that have status from Performing on reporting date 2022-12-31 and status Non Performing on reporting date 2023-03-31?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: corporate_portfolio
Thought:The corporate_portfolio table seems to be the most relevant one for this query. I should check its schema to understand its structure and the fields it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio"
Observation: 
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25

#### sql + rw

In [27]:
output = agent.run("Read the information in table corporate_portfolio for loan id 1439964799kgeq68 and for reporting date 31-12-2022")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: corporate_portfolio
Thought:The corporate_portfolio table is available in the database. Now, I need to check the schema of the corporate_portfolio table to understand its structure and the fields it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio"
Observation: 
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.7

In [28]:
output = agent.run("What will be the RW if PD doubles")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: corporate_portfolio
Thought:The corporate_portfolio table seems to be the most relevant one. I should check its schema to see if it contains the necessary columns for the RWTool.
Action: sql_db_schema
Action Input: "corporate_portfolio"
Observation: 
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0

In [3]:
# output = agent.run("What about if LGD doubles")

In [4]:
# output = agent.run("And if MATURITY becomes 6.23")

In [5]:
# output = agent.run("What if all three parameters change like that?")

In [6]:
# output = agent.run("Start over again and read the all information in table corporate_portfolio for loan id 1439964799kgeq68 but for reporting date 2023-03-31")

In [7]:
# output = agent.run("What is the change in PD for loan id 1439964799kgeq68 between reporting date 2022-12-31 and 2023-03-31")

In [8]:
# output = agent.run("Read the information for ACCOUNT_NUMBER = '1851285872eypk20' in rwa_data table and then calculate the RW if LGD doubles and PD becomes 0.2")